In [1]:
import dask
import dask.array as da
import distributed
from distributed import span, wait

dask.config.set({"optimization.fuse.active": False})
client = distributed.Client()

with span("gen_data"):
    a = da.random.random((2**15, 2**15), chunks=("auto", -1))
    a = a.persist()
    wait(a)

a

dask.array<random_sample, shape=(32768, 32768), dtype=float64, chunksize=(512, 32768), chunktype=numpy.ndarray>

In [2]:
with span("rechunk"):
    a = a.rechunk((-1, "auto"), method="p2p")
    a = a.persist()
    wait(a)

In [ ]:
metrics = client.cluster.scheduler.cumulative_worker_metrics

In [ ]:
import pandas
pandas.Series(
    {
        k[1:]: v
        for k, v in metrics.items()
        if isinstance(k, tuple) 
        and k[0] == "execute"
    }
).sort_index()

In [ ]:
pandas.Series(
    {
        k: v
        for k, v in metrics.items()
        if isinstance(k, tuple) 
        and k[0] != "execute"
    }
).sort_index()

In [ ]:
metrics["execute", "rechunk-transfer", "shuffle-shards", "bytes"] / metrics["execute", "rechunk-transfer", "shuffle-shards", "count"]